In [1]:
#PATH_data = f"../abyss_data/"
#Mount external drive: sudo mount -t drvfs D: /mnt/d
PATH_data = f"/mnt/d/abyss_data/"
PATH_GENO = f"{PATH_data}genotype"
PATH_SUMSTAT = f"{PATH_data}sumstat/mono/"
PATH_SUMSTAT_no_corr = f"{PATH_data}sumstat_no_corr"
PATH_GENO_segmented = f"{PATH_GENO}/segmented/"
PATH_GENO_mini_dim = f"{PATH_GENO}/mini_dim/"
PATH_GENO_maf = f"{PATH_GENO}/maf/"


PATH_PHENO = f"{PATH_data}phenotype/"
PATH_PHENO_ancestry = f"{PATH_PHENO}/ancestry/all.panel"
PATH_PLOTS = f"{PATH_data}plots/"
PATH_PARAMS = './hyper_params_to_test_AF.txt'


In [2]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
from collections import Counter
import os
from os import listdir
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'


2023-02-23 15:48:02.440872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 15:48:02.985018: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 15:48:02.985068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-23 15:48:03.015440: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-23 15:48:03.947896: W tensorflow/stream_executor/platform/de

In [3]:
sigmap = 3

# If you happen to have labelled population data, we can create a synthetic phenotype containing population stratification

path_humans = PATH_PHENO_ancestry
humans = pd.read_csv(path_humans, sep="\t")
humans = humans.drop(columns=['Unnamed: 5', 'Unnamed: 6'])
humans = humans.rename({'sample': 'FID', 'sample.1': 'IID'}, axis=1)
humans['ancestry'] = humans['super_pop']
humans['ancestry'].mask(humans['super_pop'] == 'AFR', 0, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'AMR', 1, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'EAS', 2, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'EUR', 3, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'SAS', 4, inplace=True)

# Make synthetic phenotype with no population stratification and no causal effect

afr = humans.loc[humans['ancestry'] == 0]
amr = humans.loc[humans['ancestry'] == 1]
eas = humans.loc[humans['ancestry'] == 2]
eur = humans.loc[humans['ancestry'] == 3]
sas = humans.loc[humans['ancestry'] == 4]

mu, sigma = 150, sigmap  # mean and standard deviation for africans
afr["no_strat_no_causal"] = np.random.normal(mu, sigma, len(afr))
afr["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for americans
amr["no_strat_no_causal"] = np.random.normal(mu, sigma, len(amr))
amr["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for east asians
eas["no_strat_no_causal"] = np.random.normal(mu, sigma, len(eas))
eas["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for europeans
eur["no_strat_no_causal"] = np.random.normal(mu, sigma, len(eur))  # 4
eur["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for south east asians
sas["no_strat_no_causal"] = np.random.normal(mu, sigma, len(sas))  # 5
sas["bias_no_strat"] = 150

# Make synthetic phenotype with population stratification and no causal effect

mu, sigma = 190, sigmap  # mean and standard deviation for africans
afr["strat_no_causal"] = np.random.normal(mu, sigma, len(afr))
afr["bias_strat"] = 180
mu, sigma = 170, sigmap  # mean and standard deviation for americans
amr["strat_no_causal"] = np.random.normal(mu, sigma, len(amr))
amr["bias_strat"] = 170
mu, sigma = 150, sigmap  # mean and standard deviation for east asians
eas["strat_no_causal"] = np.random.normal(mu, sigma, len(eas))
eas["bias_strat"] = 150
mu, sigma = 180, sigmap  # mean and standard deviation for europeans
eur["strat_no_causal"] = np.random.normal(mu, sigma, len(eur))  # 4
eur["bias_strat"] = 175
mu, sigma = 160, sigmap  # mean and standard deviation for south east asians
sas["strat_no_causal"] = np.random.normal(mu, sigma, len(sas))  # 5
sas["bias_strat"] = 160
humans = pd.concat([afr, amr, eas, eur, sas]).sort_index()


/tmp/ipykernel_2298/2697417520.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afr["no_strat_no_causal"] = np.random.normal(mu, sigma, len(afr))
/tmp/ipykernel_2298/2697417520.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afr["bias_no_strat"] = 150
/tmp/ipykernel_2298/2697417520.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [4]:
humans.to_pickle(f"{PATH_data}sumstat/humans_no_causal.pkl")
